# Parse Mueller Timeline Spreadsheet

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
import json

In [2]:
df = pd.read_csv('outline.csv')
df.head()

,YEAR,DATE,MAJOR EVENTS,INVESTIGATIONS,"RUSSIANS, HACKING",TRUMP,COHEN,MANAFORT and GATES,KUSHNER,TRUMP JR.,FLYNN,PAPADOPOULOS,PAGE,"SESSIONS, STONE AND OTHER"
0,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998,NaN,"1=HYPERLINK(""https://www.fnlondon.com/articles...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1999 - 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1999 - 2000,NaN,NaN,NaN,NaN,"1=HYPERLINK(""http://thehill.com/blogs/pundits-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1=HYPERLINK(""http://www.nationalreview.com/art..."
4,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# forward fill multi-row date categories

In [3]:
df['YEAR'] = df['YEAR'].fillna(method='ffill')
df['DATE'] = df['DATE'].fillna(method='ffill')

# Restructure

In [4]:
def parse_info(info):
    if info.startswith('1=HYPERLINK('):
        info = info[12:-1]
        index = info.find(',')
        m = [info[:index].strip('\"'), info[index+1:].strip('\"')]
        
        assert len(m) == 2, "extra comma?" + info
        return m[1], m[0]
    else:
        return info, np.nan

dict_cols = []
for i, r in df.iterrows():
    events = []
    for k, v in r.items():
        if not pd.isnull(v) and k not in ['YEAR', 'DATE']:
            events.append((k, v))
    
    for cat, info in events:
        d = {}
        d['year'] = r['YEAR']
        d['time_description'] = r['DATE']
        d['category'] = cat
        d['info'], d['url'] = parse_info(info)
        dict_cols.append(d)

In [5]:
df2 = pd.DataFrame(dict_cols)
for col in df2.columns:
    df2[col] = df2[col].str.strip()
df2.columns

Index(['category', 'info', 'time_description', 'url', 'year'], dtype='object')

# Check URLs

In [6]:
# urls = df2['url'].unique()

# import requests
# errors = []
# for i, url in enumerate(urls):
#     print(round(i/len(urls)*100, 1), end=' ')
#     try:
#         s = requests.get(url)
#         if str(s) == '<Response [404]>':
#             print(url)
#             errors.append(url)
#     except:
#         print(url)
#         errors.append(url)
        
# print(errors)

The following urls threw errors:

'https://www.ajc.com/news/the-curious-journey-carter-page-the-former-trump-adviser-who-can-stay-out-the-spotlight/EOo1COIZz9WLJTzdm7JOuL/', None, 'http://docquery.fec.gov/pdf/780/201507159000184780/201507159000184780.pdf#page=49', 'http://www.cnn.com/2015/04/23/politics/russian-hackers-pentagon-network/index.html/', 'https://intelligence.house.gov/uploadedfiles/final_russia_investigation_report.pdf', 'https://intelligence.house.gov/uploadedfiles/carter_page_hpsci_hearing_transcript_nov_2_2017.pdf', 'http://www.independent.co.uk/news/world/americas/us-politics/donald-trump-russian-ambassador-sergey-kislyak-meeting-election-campaign-a7617261.html', 'http://www.mcclatchydc.com/news/politics-government/white-house/article208870264.html', 'https://www.whitehouse.gov/the-press-office/2017/02/14/press-briefing-press-secretary-sean-spicer-2142017-12', 'https://www.whitehouse.gov/the-press-office/2017/05/18/remarks-president-trump-and-president-santos-colombia-joint-press', 'https://www.mccain.senate.gov/public/index.cfm/2017/10/senators-mccain-cardin-question-administration-delay-in-russia-sanctions-implementation', 'https://www.mccaskill.senate.gov/media-center/news-releases/mccaskill-statement-on-report-of-attempted-russian-hacking', 'https://twitter.com/realdonaldtrump/status/1031550517552603138" '

# Remove duplicate events in different categories

In [7]:
# There are repetitions of the same event in different categories
# for i, r in df2.iterrows():
#     if df2[df2['info'] == r['info']].shape[0] > 1:
#         print(r['info'])

In [8]:
cats = ['MAJOR EVENTS', 'TRUMP', 'SESSIONS, STONE AND OTHER', 'PAGE',
       'MANAFORT and GATES', 'COHEN', 'FLYNN', 'INVESTIGATIONS',
       'RUSSIANS, HACKING', 'PAPADOPOULOS', 'KUSHNER', 'TRUMP JR.']

d = {c:[] for c in cats}
for i, r in df2.iterrows():
    new_df = df2[df2['info'] == r['info']]
    these_cats = new_df.category.unique()
    for c in cats:
        if c in these_cats:
            d[c].append(True)
        else:
            d[c].append(False)
            
for c in cats:
    df2[c] = d[c]
    
# df_c = df2.drop(['category'], axis=1)
df_c = df2.copy()
# df_c = df_c.drop_duplicates()

### Condense into one category

def group_cats(r):
    return [c for c in cats if r[c]]

df_c['categories'] = df_c.apply(group_cats, axis=1)
df_n = df_c.drop(cats, axis=1)

# date handling

In [9]:
# load csv and create mapper dict
date_mapper = {}
date_mapper[np.nan] = [np.nan, np.nan]
df_map = pd.read_csv('date_encoder.csv')
for i, r in df_map.iterrows():
    date_mapper[r['text']] = [r['start'], r['end']]

In [10]:
# apply mapper to dataframe
def parse_date(x):
    return date_mapper[x]

m = df_n['time_description'].apply(lambda x: parse_date(x))
m = np.array(m.tolist())
df_n['start'] = m[:, 0]
df_n['end'] = m[:, 1]
df_n.loc[df_n['start'] == 'nan', 'start'] = np.nan
df_n.loc[df_n['end'] == 'nan', 'end'] = np.nan

In [11]:
# create 'percent through the year' column
dates_p = {m:i for i, m in enumerate([dt(2000, 1, 1) + datetime.timedelta(days=1)*j for j in range(366)])}
def date_to_perc(string):
    if not pd.isnull(string):
        m = dt.strptime(string + '-2000', '%m-%d-%Y')
        return dates_p[m] / 366 * 100
    else:
        return np.nan

df_n['start_p'] = df_n['start'].apply(date_to_perc)
df_n['width'] = df_n['end'].apply(date_to_perc) - df_n['start_p']
sum(df_n['width'] < 1)
df_n[df_n.year == '2006']

,category,info,time_description,url,year,categories,start,end,start_p,width
7,COHEN,Cohen joins the Trump Organization.,June,http://abcnews.go.com/US/meet-michael-cohen-tr...,2006,[COHEN],06-01,06-30,41.530055,7.923497
8,MANAFORT and GATES,"According to the Washington Post, Manafort sig...",June,https://www.washingtonpost.com/news/politics/w...,2006,[MANAFORT and GATES],06-01,06-30,41.530055,7.923497
9,"SESSIONS, STONE AND OTHER",Ivanka Trump and Donald Trump Jr. go to Moscow...,June,https://www.nytimes.com/2017/01/16/us/politics...,2006,"[SESSIONS, STONE AND OTHER]",06-01,06-30,41.530055,7.923497


In [12]:
# df_n[df_n['width'] < 1]['end']

# Levels

# by lone date

In [13]:
# map id to a level
df_n['ind'] = df_n.index
dict_df = {}
for y in df_n.year.unique():
    for c in cats:
        idf = f"{y}_{c}"
        dict_df[idf] = df_n[(df_n['year'] == str(y)) & (df_n['category'] == c)]

def parse_level(df):
    previous_level = 0
    lowest_percent = None
    level_dict = {}
    for i, r in df.iterrows():
        if pd.isnull(r['start']):
            level_dict[r['ind']] = 'above'
        elif not pd.isnull(r['end']):
            level_dict[r['ind']] = 'range'
        else:
            if lowest_percent is None:
                lowest_percent = r['start_p']
            else:
                if r['start_p'] < lowest_percent + 1:
                    previous_level += 1
                else:
                    lowest_percent = r['start_p']
                    previous_level = 0
            level_dict[r['ind']] = previous_level 
    return level_dict

d = {}
for k, df_sub in dict_df.items():
    d.update(parse_level(df_sub))
df_n['level'] = df_n['ind'].apply(lambda x: d[x])

# by ranges

In [14]:
def parse_level_range(df):
    previous_level = 0
    lowest_percent = None
    level_dict = {}
    for i, r in df.iterrows():
        if not pd.isnull(r['end']):
            if lowest_percent is None:
                lowest_percent = r['start_p'] + r['width']
            else:
                if r['start_p'] < lowest_percent:
                    previous_level += 1
                else:
                    lowest_percent = r['start_p']
                    previous_level = 0
            level_dict[r['ind']] = previous_level 
    return level_dict
for k, df_sub in dict_df.items():
    d.update(parse_level_range(df_sub))
df_n['level'] = df_n['ind'].apply(lambda x: d[x])
df_n[~pd.isnull(df_n['end'])].level.max()

2

# What categories are in each year?

In [15]:
d = {}
df_s = df_n[~pd.isnull(df_n['start'])]

order_cats = {'MAJOR EVENTS': '01_MAJOR',
'INVESTIGATIONS': '02_INVESTIGATIONS',
'RUSSIANS, HACKING': '03_RUSSIANS',
'TRUMP': '04_TRUMP',
'COHEN': '05_COHEN',
'MANAFORT and GATES': '06_MANAFORT',
'KUSHNER': '07_KUSHNER',
'TRUMP JR.': '08_JR',
'FLYNN': '09_FLYNN',
'PAPADOPOULOS': '10_PAPADOPOULOS',
'PAGE': '11_PAGE',
'SESSIONS, STONE AND OTHER': '12_SESSIONS',}

df_s['cat_ord'] = df_s['category'].apply(lambda x: order_cats[x])
for y in df_n.year.unique():
    d[y] = sorted(df_s[df_s.year == y].cat_ord.unique())
for y in d.keys():
    d[y] = [m.split('_')[1] for m in d[y]]
d


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'1998': [],
 '1999 - 2000': [],
 '2001': [],
 '2004': [],
 '2005': ['MANAFORT'],
 '2006': ['COHEN', 'MANAFORT', 'SESSIONS'],
 '2007': ['TRUMP', 'MANAFORT'],
 '2008': ['TRUMP'],
 '2011': ['MANAFORT'],
 '2012': ['MANAFORT'],
 '2013': ['TRUMP', 'COHEN', 'MANAFORT', 'PAGE'],
 '2014': ['MAJOR', 'INVESTIGATIONS', 'RUSSIANS', 'FLYNN', 'SESSIONS'],
 '2015': ['MAJOR',
  'INVESTIGATIONS',
  'RUSSIANS',
  'TRUMP',
  'COHEN',
  'MANAFORT',
  'FLYNN',
  'SESSIONS'],
 '2016': ['MAJOR',
  'INVESTIGATIONS',
  'RUSSIANS',
  'TRUMP',
  'COHEN',
  'MANAFORT',
  'KUSHNER',
  'JR',
  'FLYNN',
  'PAPADOPOULOS',
  'PAGE',
  'SESSIONS'],
 '2018': ['MAJOR',
  'INVESTIGATIONS',
  'RUSSIANS',
  'TRUMP',
  'COHEN',
  'MANAFORT',
  'KUSHNER',
  'JR',
  'FLYNN',
  'PAPADOPOULOS',
  'PAGE',
  'SESSIONS'],
 '2017': ['MAJOR',
  'INVESTIGATIONS',
  'RUSSIANS',
  'TRUMP',
  'COHEN',
  'MANAFORT',
  'KUSHNER',
  'JR',
  'FLYNN',
  'PAPADOPOULOS',
  'PAGE',
  'SESSIONS'],
 '2017-2018': [],
 '2019': ['MAJOR', 'RUSSIANS', 

# Category Ids

In [16]:
cat_ids = {'MAJOR EVENTS': 'MAJOR',
'RUSSIANS, HACKING': 'RUSSIANS',
'MANAFORT and GATES': 'MANAFORT',
'TRUMP JR.': 'JR',
'SESSIONS, STONE AND OTHER': 'SESSIONS',}
df_n['cat_id'] = df_n['category'].apply(lambda x: cat_ids[x] if x in cat_ids else x)

In [17]:
df_n[(df_n['year'] == '2018') & (df_n['cat_id'] == "MANAFORT")]

,category,info,time_description,url,year,categories,start,end,start_p,width,ind,level,cat_id
553,MANAFORT and GATES,"Mueller's team brings a second indictment, wit...",Feb. 22,https://www.justice.gov/file/1038391/download,2018,"[MANAFORT and GATES, INVESTIGATIONS]",02-22,NaN,14.207650,NaN,553,0,MANAFORT
555,MANAFORT and GATES,"Gates pleads guilty to two lesser charges, bec...",Feb. 23,https://www.washingtonpost.com/politics/former...,2018,[MANAFORT and GATES],02-23,NaN,14.480874,NaN,555,1,MANAFORT
556,MANAFORT and GATES,"Between this date and April 2018, Manafort and...",Feb. 23,https://www.justice.gov/file/1070326/download,2018,[MANAFORT and GATES],02-23,NaN,14.480874,NaN,556,2,MANAFORT
580,MANAFORT and GATES,Reuters reports that Manafort's ex-son-in-law ...,May 17,https://www.reuters.com/article/us-usa-trump-r...,2018,[MANAFORT and GATES],05-17,NaN,37.431694,NaN,580,0,MANAFORT
583,MANAFORT and GATES,"According to prosecutors, Manafort sends a tex...",May 26,https://assets.documentcloud.org/documents/545...,2018,[MANAFORT and GATES],05-26,NaN,39.890710,NaN,583,0,MANAFORT
590,MANAFORT and GATES,Prosecutors with Mueller's team filed court do...,June 4,https://assets.documentcloud.org/documents/449...,2018,[MANAFORT and GATES],06-4,NaN,42.349727,NaN,590,0,MANAFORT
595,MANAFORT and GATES,Mueller drops a new obstruction of justice ind...,June 8,https://www.nytimes.com/2018/06/08/us/politics...,2018,"[MANAFORT and GATES, INVESTIGATIONS]",06-8,NaN,43.442623,NaN,595,0,MANAFORT
599,MANAFORT and GATES,"Judge orders Manafort to jail to await trial, ...",June 15,https://www.washingtonpost.com/local/public-sa...,2018,[MANAFORT and GATES],06-15,NaN,45.355191,NaN,599,0,MANAFORT
631,MANAFORT and GATES,Manafort is found guilty of eight counts of f...,August 21,https://www.theguardian.com/us-news/2018/aug/2...,2018,[MANAFORT and GATES],08-21,NaN,63.661202,NaN,631,0,MANAFORT
639,MANAFORT and GATES,Manafort agrees to plead guilty to two counts ...,Sept. 14,https://www.washingtonpost.com/world/national-...,2018,[MANAFORT and GATES],09-14,NaN,70.218579,NaN,639,0,MANAFORT


In [18]:
dates_p = {m:i for i, m in enumerate([dt(2000, 1, 1) + datetime.timedelta(days=1)*j for j in range(366)])}
l = [dates_p[dt(2000, i, 1)] for i in range(1, 13)]
l = [round(i / 366 * 100, 2) for i in l]
l.append(100)
l = [round(l[i+1] - l[i], 2) for i, m in enumerate(l[:-1])]
l

[8.47, 7.92, 8.47, 8.2, 8.47, 8.2, 8.47, 8.47, 8.19, 8.47, 8.2, 8.47]

# Sloppy exception handling

In [30]:
if '2017-2018' in df_n['year']:
    i = df_n[df_n['year'] == '2017-2018']['ind'].tolist()
    df_n.loc[df_n['ind'] == i, 'year'] = '2018'
    df_n.loc[df_n['ind'] == i, 'time_description'] = 'Unknown dates, over 2017-2018'
    df_n[df_n['ind'] == i]
# df_n.info = df_n['info'].str.replace("\"", "  ", regex=False)sst
m = df_n[df_n['url'] == 'https://www.washingtonpost.com/politics/michael-cohen-trumps-former-lawyer-pleads-guilty-to-lying-to-congress/2018/11/29/5fac986a-f3e0-11e8-bc79-68604ed88993_story.html?utm_term=.e61ca545bd15']
m['info'][654]

'Trump says Cohen is *now* lying about the Moscow project to ""get a reduced sentence.'

In [20]:
change_to_one = ["https://www.wsj.com/articles/former-trump-adviser-paul-manaforts-bank-records-sought-in-probe-1494637248",
"https://www.boston.com/news/politics/2018/07/16/russian-hackers-used-us-online-infrastructure-against-itself",
"https://www.justice.gov/file/1007346/download",
"http://www.cnn.com/2017/08/28/politics/russia-trump-campaign-clay-dearborn-request/index.html"]

change_to_two = ["https://int.nyt.com/data/documenthelper/586-roger-stone-indictment/d34c762c3e142f844c2b/optimized/full.pdf#page=1",
"http://www.cnn.com/2017/06/21/politics/russia-hacking-hearing-states-targeted/index.html"]

for i, r in df_n.iterrows():
    if r['url'] in change_to_one:
        df_n.at[i, 'level'] = 1
    if r['url'] in change_to_two:
        df_n.at[i, 'level'] = 2


# OUTPUT

In [21]:
df_o = df_n[df_n['year'] != '2017-2018']
df_o.to_json('clean.json', orient='index')